In [2]:
import json

# --- 1. Datos iniciales (sin cambios) ---
dict_1 = {'id_A': {'doc_01': 0.98}, 'id_B': {'doc_02': 0.87}}
dict_2 = {'id_A': {'doc_01': 0.95}, 'id_B': {'doc_02': 0.91}}
dict_3 = {'id_A': {'doc_03': 0.85}, 'id_B': {'doc_02': 0.93}}
dict_4 = {'id_A': {'doc_07': 0.99}, 'id_B': {'doc_08': 0.76}}

lista_de_diccionarios = [dict_1, dict_2, dict_3, dict_4]

# --- 2. Contar las repeticiones (sin cambios) ---
doc_counts = {}
for d in lista_de_diccionarios:
    for id_clave, docs in d.items():
        if id_clave not in doc_counts:
            doc_counts[id_clave] = {}
        for doc_id in docs.keys():
            doc_counts[id_clave][doc_id] = doc_counts[id_clave].get(doc_id, 0) + 1

# --- 3. Ordenar y crear el diccionario final (AQUÍ ESTÁ EL CAMBIO) ---
# Se crea el diccionario final que contendrá los resultados.
sorted_doc_counts_dict = {}

# Se itera sobre el diccionario de conteos.
for id_clave, counts in doc_counts.items():
    # Primero, se obtiene la lista ordenada de tuplas (doc_id, conteo).
    sorted_docs_list = sorted(counts.items(), key=lambda item: item[1], reverse=True)
    
    # ¡NUEVO! Se convierte esa lista ordenada de nuevo en un diccionario.
    # Python 3.7+ mantiene el orden de inserción, por lo que el diccionario quedará ordenado.
    ordered_dict = dict(sorted_docs_list)
    
    # Se almacena el nuevo diccionario ordenado en el resultado final.
    sorted_doc_counts_dict[id_clave] = ordered_dict

# --- 4. Imprimir el resultado final ---
print(json.dumps(sorted_doc_counts_dict, indent=2))

{
  "id_A": {
    "doc_01": 2,
    "doc_03": 1,
    "doc_07": 1
  },
  "id_B": {
    "doc_02": 3,
    "doc_08": 1
  }
}


In [2]:
import json

def reciprocal_rank_fusion(results: list[list], k=60):
    """ 
    Reciprocal Rank Fusion que toma múltiples listas de documentos clasificados
    y un parámetro opcional k utilizado en la fórmula RRF.
    Esta versión no tiene dependencias externas como LangChain.
    """
    
    # Inicializa un diccionario para mantener los puntajes fusionados de cada documento único
    fused_scores = {}

    # Itera a través de cada lista de documentos clasificados
    for docs in results:
        # Itera a través de cada documento en la lista, con su rango (posición)
        for rank, doc in enumerate(docs):
            # Convierte el documento a un formato de string para usarlo como clave.
            # Usamos json.dumps con sort_keys=True para asegurar que el mismo
            # diccionario siempre produzca el mismo string.
            doc_str = json.dumps(doc, sort_keys=True)
            
            # Si el documento aún no está en el diccionario fused_scores, añádelo con un puntaje inicial de 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            
            # Actualiza el puntaje del documento usando la fórmula RRF: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Ordena los documentos basándose en sus puntajes fusionados en orden descendente
    reranked_results = [
        (json.loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Devuelve los resultados re-clasificados como una lista de tuplas (documento, puntaje)
    return reranked_results

In [5]:
# Documentos de ejemplo (pueden ser cualquier objeto serializable a JSON)
doc1 = {"id": "A"}
doc2 = {"id": "B"}
doc3 = {"id": "C"}

# Listas de resultados de dos sistemas de búsqueda diferentes
results_list_1 = [doc1, doc2]
results_list_2 = [doc2, doc3, doc1]

# Combinar las listas usando RRF
final_ranking = reciprocal_rank_fusion([results_list_1, results_list_2], k=60)

# Imprimir el ranking final
for doc, score in final_ranking:
    print(f"Puntaje: {score:.4f} -> Documento: {doc}")

# Salida esperada:
# Puntaje: 0.0331 -> Documento: {'id': 'B', 'text': 'Contenido del documento B'}
# Puntaje: 0.0328 -> Documento: {'id': 'A', 'text': 'Este es el documento A'}
# Puntaje: 0.0164 -> Documento: {'id': 'C', 'text': 'Información sobre C'}

Puntaje: 0.0331 -> Documento: {'id': 'B'}
Puntaje: 0.0328 -> Documento: {'id': 'A'}
Puntaje: 0.0164 -> Documento: {'id': 'C'}


In [6]:
print(final_ranking)

[({'id': 'B'}, 0.03306010928961749), ({'id': 'A'}, 0.03279569892473118), ({'id': 'C'}, 0.01639344262295082)]


In [7]:
def reciprocal_rank_fusion_from_scores(results: list[dict], k=60):
    """
    Realiza Reciprocal Rank Fusion a partir de una lista de diccionarios.
    Cada diccionario contiene pares {id_documento: puntaje}.
    """
    
    # Diccionario para mantener los puntajes RRF acumulados de cada documento
    fused_scores = {}

    # Itera a través de cada diccionario de resultados (de cada sistema de búsqueda)
    for result_dict in results:
        
        # --- PASO CLAVE: Convertir puntajes en una lista ordenada por rango ---
        # Ordenamos los items del diccionario por su valor (puntaje) en orden descendente.
        # Esto nos da una lista de tuplas (id_doc, score) ordenada por relevancia.
        ranked_list = [
            doc_id for doc_id, score in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)
        ]
        
        # --- Ahora aplicamos la lógica RRF original sobre la lista ordenada ---
        # Itera a través de la lista recién creada para obtener el rango y el id del documento
        for rank, doc_id in enumerate(ranked_list):
            
            # Si es la primera vez que vemos este doc_id, lo inicializamos
            if doc_id not in fused_scores:
                fused_scores[doc_id] = 0
            
            # Sumamos el puntaje RRF basado en el rango
            fused_scores[doc_id] += 1 / (rank + k)

    # Ordenamos el resultado final basado en el puntaje RRF acumulado
    reranked_results = sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)

    # Devolvemos la lista final de tuplas (id_documento, puntaje_rrf_final)
    return reranked_results

In [8]:
# Resultados del Sistema 1 (doc_A es el mejor)
resultados_1 = {'doc_A': 0.9, 'doc_C': 0.5}

# Resultados del Sistema 2 (doc_B es el mejor)
resultados_2 = {'doc_B': 0.8, 'doc_A': 0.7, 'doc_C': 0.6}

# Lista de todos los resultados
todas_las_listas = [resultados_1, resultados_2]

# Ejecutar la fusión
ranking_final = reciprocal_rank_fusion_from_scores(todas_las_listas, k=60)

# Imprimir el resultado
for doc_id, score in ranking_final:
    print(f"Puntaje RRF: {score:.4f} -> Documento: {doc_id}")

Puntaje RRF: 0.0331 -> Documento: doc_A
Puntaje RRF: 0.0325 -> Documento: doc_C
Puntaje RRF: 0.0167 -> Documento: doc_B


In [9]:
ranking_final

[('doc_A', 0.03306010928961749),
 ('doc_C', 0.03252247488101534),
 ('doc_B', 0.016666666666666666)]

In [10]:
def reciprocal_rank_fusion_from_scores(results: list[dict], k=60) -> dict:
    """
    Realiza Reciprocal Rank Fusion y devuelve un diccionario ordenado
    con los resultados {id_documento: puntaje_rrf_final}.
    """
    
    # Diccionario para mantener los puntajes RRF acumulados de cada documento
    fused_scores = {}

    # Itera a través de cada diccionario de resultados
    for result_dict in results:
        
        # 1. Convertir puntajes en una lista ordenada por rango
        ranked_list = [
            doc_id for doc_id, score in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)
        ]
        
        # 2. Aplicar la lógica RRF sobre la lista ordenada
        for rank, doc_id in enumerate(ranked_list):
            if doc_id not in fused_scores:
                fused_scores[doc_id] = 0
            fused_scores[doc_id] += 1 / (rank + k)

    # 3. Ordenar los resultados y convertirlos de nuevo a un diccionario
    # Primero, obtenemos una lista de tuplas ordenada por el puntaje RRF
    sorted_items = sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Luego, convertimos esa lista ordenada en un nuevo diccionario
    # En Python 3.7+ este diccionario mantendrá el orden
    reranked_dict = dict(sorted_items)

    return reranked_dict

In [11]:
# Resultados del Sistema 1
resultados_1 = {'doc_A': 0.9, 'doc_C': 0.5}

# Resultados del Sistema 2
resultados_2 = {'doc_B': 0.8, 'doc_A': 0.7, 'doc_C': 0.6}

# Lista de todos los resultados
todas_las_listas = [resultados_1, resultados_2]

# Ejecutar la fusión
ranking_final_dict = reciprocal_rank_fusion_from_scores(todas_las_listas, k=60)

# Imprimir el diccionario resultante
print("Diccionario de ranking final:")
print(ranking_final_dict)

# Puedes iterar sobre él y ver que el orden se mantiene
print("\nIterando sobre el diccionario ordenado:")
for doc_id, score in ranking_final_dict.items():
    print(f"Puntaje RRF: {score:.4f} -> Documento: {doc_id}")

Diccionario de ranking final:
{'doc_A': 0.03306010928961749, 'doc_C': 0.03252247488101534, 'doc_B': 0.016666666666666666}

Iterando sobre el diccionario ordenado:
Puntaje RRF: 0.0331 -> Documento: doc_A
Puntaje RRF: 0.0325 -> Documento: doc_C
Puntaje RRF: 0.0167 -> Documento: doc_B
